<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_DMM/blob/main/Conv2D_LSTM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
import tensorflow as tf
import pandas as pd
import numpy as np

tf.random.set_seed(1)

#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
train_data_input = df_train.iloc[:,:-1]
train_data_output = df_train.iloc[:,-1]
test_data_input = df_test.iloc[:,:-1]
test_data_output = df_test.iloc[:,-1]

train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]

trainSet_for_result = train_data
testSet_for_result = test_data

print(train_data_input.shape,train_data_input)
print(train_data_output.shape, train_data_output)
print("train_data : ",train_data.shape,train_data)
print("test_data : ",test_data.shape,test_data)

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  nume = data-np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min):
  diff = max-min
  back = data * diff + min
  return back

seqLength = 10
dataDim = df_test.shape[1]
outputDim = 1
lr = 0.001
iterations = 20
batch_size = 32

dataDim = train_data_input.shape[1]

print(dataDim)
test_data_input_max = np.max(test_data_input)
train_data_input_max = np.max(train_data_input)
test_data_output_max = np.max(test_data_output)
train_data_output_max = np.max(train_data_output)

test_data_input_min = np.min(test_data_input)
train_data_input_min = np.min(train_data_input)
test_data_output_min = np.min(test_data_output)
train_data_output_min = np.min(train_data_output)

print(test_data_output_max)
print(test_data_output_min)

train_data_input = MinMaxScaler(train_data_input)
train_data_output = MinMaxScaler(train_data_output)
test_data_input = MinMaxScaler(test_data_input)
test_data_output = MinMaxScaler(test_data_output)

train_data = MinMaxScaler(train_data)
test_data = MinMaxScaler(test_data)

train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]
print(train_data)
print(test_data)

trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
testSize = int(len(test_data))
testSet = test_data[0:testSize]
testSet_for_result = testSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)
testSet_max = np.max(testSet)
testSet_min = np.min(testSet)
trainSet_max = (trainSet_max[dataDim-1])
trainSet_min = (trainSet_min[dataDim-1])
testSet_max = (testSet_max[dataDim])
testSet_min = (testSet_min[dataDim])


trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

print(trainSet.shape) # (5807,5)
print(testSet.shape) # (10882,5)

print(testSet_max)
print(testSet_min)



In [ ]:
# Data Dimension matching for CNN
print(train_data_input.shape,type(train_data_input))
train_data_input_np = train_data_input.to_numpy()
print(train_data_input_np.shape,type(train_data_input_np))
train_data_input_np2 = train_data_input_np.reshape((train_data_input_np.shape[0],train_data_input_np.shape[1],1))
train_data_output_np2 = train_data_output.to_numpy()
print(train_data_input_np2.shape)
print(train_data_output_np2.shape)

In [ ]:
#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length): 
        print(i)
        _x = time_series.iloc[i:i + seq_length, : ]
        _y = time_series.iloc[i + seq_length, ]  
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = sliding_window(trainSet, seqLength)
testX, testY = sliding_window(testSet, seqLength)
print("Finished")

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

# Hyper Parameter
filters = 4
kernel_size = (2,2)
pooling_size = (2,2)
stride = (1,1)
input_shape = train_data.shape[1]
outputDim = input_shape
hiddenDim = 20
initializer = tf.keras.initializers.GlorotNormal()
After_Conv = 56 # Check before Run with model summary

#LSTM after CNN
model = keras.Sequential()
model.add(Conv2D(filters = filters, kernel_size = kernel_size, strides = stride, activation = 'relu', input_shape = (seqLength,input_shape,1)))
model.add(Conv2D(filters = filters, kernel_size = kernel_size, strides = stride, activation = 'relu',))
model.add(Conv2D(filters = filters, kernel_size = kernel_size, strides = stride, activation = 'relu',))
model.add(Flatten())
model.add(layers.Reshape((After_Conv, 1)))
model.add(layers.LSTM(units = hiddenDim, activation='tanh',input_shape = (None,After_Conv, 1), return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'tanh',return_sequences = False, kernel_initializer = initializer))
model.add(layers.Dense(outputDim, activation='linear'))
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
#Training
print(trainX.shape)
print(trainY.shape)
hist = model.fit(trainX,trainY, epochs = iterations, verbose = 0)

In [ ]:
#Evaluating
print(input_shape)
batch_size = 16

#res = model.evaluate(trainX, trainY, batch_size = batch_size)
print("Accuracy : ",res)
print(testSet_max)
print(testSet_min)
estimated = model.predict(testX, verbose = 0)
estimated = back_MinMax(estimated, testSet_max, testSet_min)
estimated = estimated[:,-1]
real = testSet_for_result.iloc[:,-1] # estimated + seqLength

print("Estimated : ",estimated.shape, estimated)
print("Real : ",real.shape, real)

In [ ]:
#Visualization
import matplotlib.pyplot as plt

plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by Conv2D,LSTM Mixture Model")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

print("Estimated : ",estimated.shape,estimated)
print("Real : ",real.shape, real)